In [4]:
!pip install transformers datasets scikit-learn -q

In [5]:
!pip install -U transformers

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer
import torch

In [6]:
df = pd.read_csv("esewa_reviews.csv")
df.head()

,Unnamed: 0,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,clean_text,tokens
0,0,bf9a3185-608d-4293-bfea-3f5157e3b426,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,AjayDas,5,1,NaN,2022-08-15 21:16:51,"Hello Ajay Sir, thank you so much for your enc...",2022-08-16 09:35:32,NaN,ajaydas,['ajaydas']
1,1,7e8ef74b-dbaa-4afd-8174-2d1ecd194b8c,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,"Useless, very slow",1,0,3.9.0.1,2022-08-15 18:52:33,"Hello Biyaz Sir, we apologize for the inconven...",2022-08-16 09:55:07,3.9.0.1,useless very slow,"['useless', 'very', 'slow']"
2,2,4cba7404-d8b4-48c9-ba5b-8a0b14764a2f,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Such a poor app. Transaction limit got decreas...,1,0,NaN,2022-08-15 18:46:52,"Hello there, we highly regret the inconvenienc...",2022-08-16 15:14:30,NaN,such a poor app transaction limit got decrease...,"['such', 'a', 'poor', 'app', 'transaction', 'l..."
3,3,ac7b0fa4-94b9-40e9-9d57-ea7f3273406b,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Not update,5,0,NaN,2022-08-15 17:05:16,"Hello Sashikant Sir, thank you so much for you...",2022-08-16 09:54:11,NaN,not update,"['not', 'update']"
4,4,0339aa75-24d4-41cb-8471-a69b39e9f3ae,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Totally frustrating application. Can't even lo...,1,12,3.9.0.1,2022-08-15 15:40:34,"Hello there, we understand your frustration he...",2022-08-15 16:25:53,3.9.0.1,totally frustrating application cant even logi...,"['totally', 'frustrating', 'application', 'ca'..."


In [7]:
df = df.dropna(subset=['clean_text'])

In [8]:
def map_sentiment(score):
  if score <=2:
    return 0
  elif score == 3:
    return 1
  else:
    return 2

In [9]:
df['sentiment'] = df['score'].apply(map_sentiment)

In [10]:
df_subset = df[['clean_text', 'sentiment']].sample(n=5000, random_state=42)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_subset['clean_text'].tolist(),
    df_subset['sentiment'].tolist(),
    test_size=0.2,
    stratify=df_subset['sentiment'],
    random_state=42
)

In [11]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [12]:
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels
})
val_dataset = Dataset.from_dict({
    'input_ids': val_encodings['input_ids'],
    'attention_mask': val_encodings['attention_mask'],
    'labels': val_labels
})

In [13]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=500,
    eval_steps=500,
)

In [15]:
!pip install wandb

In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

trainer.train()

<ipython-input-16-72729e99ec4d>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shrestha-07ayush (shrestha-07ayush-esewa-pvt) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,0.939300
20,0.749200
30,0.837700
40,0.732700
50,0.547300
60,0.559200
70,0.717000
80,0.447700
90,0.530600
100,0.574400


Step,Training Loss
10,0.939300
20,0.749200
30,0.837700
40,0.732700
50,0.547300
60,0.559200
70,0.717000
80,0.447700
90,0.530600
100,0.574400


TrainOutput(global_step=1500, training_loss=0.4736119295756022, metrics={'train_runtime': 20627.3362, 'train_samples_per_second': 0.582, 'train_steps_per_second': 0.073, 'total_flos': 1122343172496000.0, 'train_loss': 0.4736119295756022, 'epoch': 3.0})

In [ ]:
!pip install nbstripout

from google.colab import files
uploaded = files.upload()

filename = list(uploaded.keys())[0]

!nbstripout {sentiment_analysis.ipynb}

files.download(sentiment_analysis.ipynb)


In [17]:
preds = trainer.predict(val_dataset)
y_pred = preds.predictions.argmax(-1)
print(classification_report(val_labels, y_pred, target_names=["Negative", "Neutral", "Positive"]))

              precision    recall  f1-score   support

    Negative       0.82      0.78      0.80       379
     Neutral       0.26      0.16      0.20        51
    Positive       0.85      0.91      0.88       570

    accuracy                           0.82      1000
   macro avg       0.64      0.62      0.62      1000
weighted avg       0.81      0.82      0.81      1000

